# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
import copy
import math
import random
import time
from collections import deque,namedtuple

import gym
import matplotlib.pyplot as plt
import numpy as np
import progressbar as pb
import torch
import torch.nn as nn
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from unityagents import UnityEnvironment
from apex import amp

%matplotlib inline

In [2]:
# check which device is being used.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using device: ",device)

using device:  cuda:0


Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [3]:
# env = UnityEnvironment(file_name='./Reacher_Linux/Reacher.x86_64')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# # get the default brain
# brain_name = env.brain_names[0]
# brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [5]:
# # reset the environment
# env_info = env.reset(train_mode=True)[brain_name]

# # number of agents
# num_agents = len(env_info.agents)
# print('Number of agents:', num_agents)

# # size of each action
# action_size = brain.vector_action_space_size
# print('Size of each action:', action_size)

# # examine the state space 
# states = env_info.vector_observations
# state_size = states.shape[1]
# print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
# print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [6]:
# env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
# states = env_info.vector_observations                  # get the current state (for each agent)
# scores = np.zeros(num_agents)                          # initialize the score (for each agent)
# start = time.time()
# duration = 5
# while time.time() < start + duration:
#     actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#     actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#     env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#     next_states = env_info.vector_observations         # get next state (for each agent)
    
#     rewards = env_info.rewards                         # get reward (for each agent)
#     dones = env_info.local_done                        # see if episode finished
#     scores += env_info.rewards                         # update the score (for each agent)
#     print("\r{}".format(states.shape), end="")
#     states = next_states                               # roll over states to next time step
#     if np.any(dones):                                  # exit loop if episode finished
#         break
# print("\r{}".format(states))
# print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))


When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [7]:
class Actor(nn.Module):

    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        
        num_hidden1 = 256
        num_hidden2 = 128

        self.fc1 = nn.Linear(state_size, num_hidden1)
        self.fc2 = nn.Linear(num_hidden1, num_hidden2)
        self.fc3 = nn.Linear(num_hidden2, action_size)
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, state):
        x = F.relu(self.fc1(state))
#         x = self.dropout(x)
        x = F.relu(self.fc2(x))
        return torch.tanh(self.fc3(x))


In [8]:
class Critic(nn.Module):

    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        
        num_hidden1 = 256
        num_hidden2 = 128

        self.fc1 = nn.Linear(state_size, num_hidden1)
        self.fc2 = nn.Linear(num_hidden1 + action_size, num_hidden2)
        self.fc3 = nn.Linear(num_hidden2, 1)
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, state, action):
        xs = F.relu(self.fc1(state))
        x = torch.cat((xs, action), dim=1)
#         x = self.dropout(x)
        x = F.relu(self.fc2(x))
        return self.fc3(x) 


In [9]:

class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.2):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for i in range(len(x))])
        self.state = x + dx
        return self.state

In [10]:
class DDPGAgentParams():
    def __init__(
        self, 
        buffer_size=int(1e6), 
        batch_size=64, 
        actor_lr = 1e-4, 
        critic_lr = 1e-3, 
        gamma = .99, 
        tau = 1e-3,
        critic_weight_decay=1e-4):
        
        self.buffer_size = buffer_size
        self.batch_size = batch_size
        self.actor_lr = actor_lr
        self.critic_lr = critic_lr
        self.gamma = gamma
        self.tau = tau
        self.critic_weight_decay = critic_weight_decay

        


class DDPGAgent():
    def __init__(self, state_size, action_size, params, device="cpu", random_seed=123):
        self.params = params
        self.device = device
        self.seed = random.seed(random_seed)

        
        self.actor_local = Actor(state_size, action_size).to(device)
        self.actor_target = Actor(state_size, action_size).to(device)
        self.actor_optimizer = optim.Adam(self.actor_local.parameters(), lr=params.actor_lr)

        self.critic_local = Critic(state_size, action_size).to(device)
        self.critic_target = Critic(state_size, action_size).to(device)
        self.critic_loss = nn.MSELoss()
        self.critic_optimizer = optim.Adam(
            self.critic_local.parameters(), 
            lr=params.critic_lr, 
            weight_decay=params.critic_weight_decay)
        
        self.memory = ReplayBuffer(params.buffer_size, params.batch_size, random_seed)
        self.noise = OUNoise(action_size, random_seed)

    
    def step(self, state, action, reward, next_state, done):
        self.memory.add(state, action, reward, next_state, done)

        if len(self.memory) > self.params.batch_size:
            experiences = self.memory.sample()
            self.learn(experiences)
            

    def to_float_tensor(self, ndarray):
        return torch.from_numpy(ndarray).float().to(self.device)
    
    def to_numpy(self, tensor):
        return tensor.detach().cpu().numpy()
    
    def act(self, state, add_noise=True):
        state_tensor = self.to_float_tensor(state)
        self.actor_local.eval()
        with torch.no_grad():
            action = self.to_numpy(self.actor_local(state_tensor))
        
        self.actor_local.train()
        
        if add_noise:
            action += self.noise.sample()

        action = np.clip(action, -1, 1)
        return action

    def reset(self):
        self.noise.reset()
      
    
    def learn(self, experiences):
        states_tensor, actions_tensor, rewards_tensor, next_states_tensor, dones_tensor = experiences
        gamma = self.params.gamma
        
        # Q Target (with target network)
        actions_next_tensor = self.actor_target(next_states_tensor)
        Q_targets_next = self.critic_target(next_states_tensor, actions_next_tensor)
        Q_targets = rewards_tensor + (gamma * Q_targets_next * (1 - dones_tensor))

        # Q Expected (with local network)
        Q_expected = self.critic_local(states_tensor, actions_tensor)
        critic_loss = self.critic_loss(Q_expected, Q_targets)
        
        # Optimize Critic
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Actor loss
        actions_tensor = self.actor_local(states_tensor)
        actor_loss = -self.critic_local(states_tensor, actions_tensor).mean()

        # Optimize Actor
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # Soft update
        self.soft_update(self.critic_local, self.critic_target, self.params.tau)
        self.soft_update(self.actor_local, self.actor_target, self.params.tau)             
            
    def soft_update(self, local_model, target_model, tau):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)


In [11]:
class ReplayBuffer:
    def __init__(self, buffer_size, batch_size, seed=123):
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", "state, action, reward, next_state, done")
        np.random.seed(seed)

    def add(self, state, action, reward, next_state, done):
        e =self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
        
    def sample(self):
        n = min(self.batch_size, len(self.memory))
        experiences = random.sample(self.memory, k=n)
        
        states = torch.from_numpy(np.array([e.state for e in experiences])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences])).float().to(device)
        
        return states, actions, rewards, next_states, dones

    def __len__(self):
        return len(self.memory)

In [ ]:
env = UnityEnvironment(file_name='./Reacher_Linux/Reacher.x86_64')
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=True)[brain_name]

states = env_info.vector_observations
state_size = states.shape[1]
action_size = brain.vector_action_space_size

params = DDPGAgentParams()
agent = DDPGAgent(state_size, action_size, params, device=device)

# agent.actor_local.load_state_dict(torch.load('checkpoint_actor1.pth'))
# agent.critic_local.load_state_dict(torch.load('checkpoint_critic1.pth'))


episode = 1000

def step(env, action):
    env_info = env.step(action)[brain_name]
    next_state = env_info.vector_observations[0]
    reward = env_info.rewards[0]
    done = env_info.local_done[0]
    return next_state, reward, done

scores_deque = deque(maxlen=100)
max_t = 1000
scores = []
for i in range(1, episode+1):
    env_info = env.reset(train_mode=True)[brain_name]
    state = env_info.vector_observations[0]
    score = 0
    for _ in range(max_t):
        action = agent.act(state)
        next_state, reward, done = step(env, action)
        agent.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
        if done:
            break 
            
    scores_deque.append(score)
    scores.append(score)
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(i, np.mean(scores_deque)), end="")
    torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
    torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
    if i % 100 == 0:
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i, np.mean(scores_deque)))


fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Episode 13	Average Score: 0.70

In [ ]:
torch.save(agent.actor_local.state_dict(), 'checkpoint_actor1.pth')
torch.save(agent.critic_local.state_dict(), 'checkpoint_critic1.pth')


In [ ]:
# Print best agent
state = env.reset()
for t in range(200):
    action = agent.act(state)

    env.render()
    print("\r{} {}".format(t, action), end="")
    state, reward, done, _ = env.step(action)
    if done:
        break 

env.close()